In [1]:
import sys
sys.path.insert(0, "..")

import torch
import numpy as np
import torch.nn as nn
from tqdm import trange
import torch.nn.functional as F
import matplotlib.pyplot as plt

from edunets import functional, losses
from edunets.tensor import Tensor

### Basic Cross-correlation

Let's make sure the cross correlation works

In [2]:
a = np.array([
    [1., 2., 3., 0.],
    [4., 5., 6., 0.],
    [7., 8., 9., 0.],
    [-1., -1., -1., -1.]
])
v = np.array([
    [2., 3.],
    [3., 2.]
])

In [3]:
# Pytorch only supports 3 dimentional vectors for F.conv so we add dummy dimensions
x = torch.tensor(a, dtype=float)[None, None]
w = torch.tensor(v, dtype=float)[None, None]
F.conv2d(x, w)

tensor([[[[30., 40., 24.],
          [60., 70., 39.],
          [33., 38., 13.]]]], dtype=torch.float64)

In [4]:
Tensor(a).correlate(v)

tensor([[30., 40., 24.],
       [60., 70., 39.],
       [33., 38., 13.]], dtype=float32)

Now let's see if the gradients match

In [5]:
x = torch.tensor(a[None, None], dtype=float, requires_grad=True)
w = torch.tensor(v[None, None], dtype=float, requires_grad=True)
out = F.conv2d(x, w)
print(out)
out.backward(torch.ones(out.shape))

print(x.grad)
print(w.grad)

tensor([[[[30., 40., 24.],
          [60., 70., 39.],
          [33., 38., 13.]]]], dtype=torch.float64,
       grad_fn=<ThnnConv2DBackward>)
tensor([[[[ 2.,  5.,  5.,  3.],
          [ 5., 10., 10.,  5.],
          [ 5., 10., 10.,  5.],
          [ 3.,  5.,  5.,  2.]]]], dtype=torch.float64)
tensor([[[[45., 33.],
          [36., 25.]]]], dtype=torch.float64)


In [6]:
x = Tensor(a[None, None], requires_grad=True)
w = Tensor(v[None, None], requires_grad=True)
out = x.correlate(w)
print(out)
out.backward(Tensor.ones(*out.shape))

print(x.grad)
print(w.grad)

tensor([[[[30., 40., 24.],
         [60., 70., 39.],
         [33., 38., 13.]]]], dtype=float32)
[[[[ 2.  5.  5.  3.]
   [ 5. 10. 10.  5.]
   [ 5. 10. 10.  5.]
   [ 3.  5.  5.  2.]]]]
[[[[45. 33.]
   [36. 25.]]]]


They do yay! Now we have to try and add the other paramaters that are in torch's cond1d, like padding and stride.

### Cross-correlation with a stride

Let's start with stride, meaning how much the kernel shifts when performing cross-correlation. Default is 1, so the kernel just slides naturaly accross the input. 

With stride = 1, we get, for input=[1,2,3,4,5,6] and kernel=[6,7]:

6*1 + 7*2 = 20

6*2 + 7*3 = 33


...

6*4 + 7*5 = 59

so the convolution is [20, 33, 46, 59]

With stride = 2 we just take the index that are equal to 0 modulo 2
this of course generelizes to any value of stride

In [9]:
stride = 1

x = torch.tensor(a[None, None], dtype=float, requires_grad=True)
w = torch.tensor(v[None, None], dtype=float, requires_grad=True)
out = F.conv1d(x, w, stride=stride)
print(out)
out.backward(torch.ones(out.shape))

print(x.grad)
print(w.grad)

tensor([[[[30., 40., 24.],
          [60., 70., 39.],
          [33., 38., 13.]]]], dtype=torch.float64,
       grad_fn=<ThnnConv2DBackward>)
tensor([[[[ 2.,  5.,  5.,  3.],
          [ 5., 10., 10.,  5.],
          [ 5., 10., 10.,  5.],
          [ 3.,  5.,  5.,  2.]]]], dtype=torch.float64)
tensor([[[[45., 33.],
          [36., 25.]]]], dtype=torch.float64)


In [8]:
x = Tensor(a[None, None], dtype=float, requires_grad=True)
w = Tensor(v[None, None], dtype=float, requires_grad=True)
out = x.correlate(w)

# selecting base on stride
if stride > 1:
    flatten = out.reshape((-1,))

    idx = [i for i in range(flatten.shape[0]) if i % stride == 0]

    print([(i,i % stride == 0) for i in range(flatten.shape[0])])

    print(out.reshape((-1,)))
    
    print(out.reshape((-1,))[idx])
    
    out = out.reshape((-1,))[idx].reshape(tuple(1 for _ in range(len(out.shape))))

print(out)
out.backward(Tensor.ones(*out.shape))

print(x.grad)
print(w.grad)
out.graph

[(0, True), (1, False), (2, True), (3, False), (4, True), (5, False), (6, True), (7, False), (8, True)]
tensor([30., 40., 24., 60., 70., 39., 33., 38., 13.], dtype=float32)
tensor([30., 24., 70., 33., 13.], dtype=float32)


ValueError: cannot reshape array of size 5 into shape (1,1,1,1)

### Convolution Layer

The first step is to implement a convolution layer class

In [ ]:
class Conv2d:
    def __init__(self, input_shape, kernel_size, depth):
        input_depth, input_height, input_width = input.shape
        
        self.depth = depth
        self.input_shape = input_shape
        self.input_depth = input_depth
        self.output_shape = (depth, input_height - kernel_size + 1, input_width - kernel_size + 1)
        self.kernels_shape = (depth, input_depth, kernel_size, kernel_size)
        
        self.kernels = Tensor.uniform(*self.kernels_shape)
        self.biases = Tensor.uniform(*self.output_shape)

    def forward(self, x):
        output = self.biases.copy()

        for i in range(self.depth):
            for j in range(self.input_depth):
                output[i] = output[i] + x[j].correlate(self.kernels[i,j], mode="valid")